In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input, Activation
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as Kb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt4
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (15,15)
import pylab 
import time
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
df_loc = pd.read_csv("../synthetic_data_simulations/2d_gaussian_1200_1.csv", sep = ",")
# df_loc = pd.read_csv("../2D_biv_matern_large.csv", sep = ",")

N = len(df_loc)
s = np.vstack((df_loc["x"],df_loc["y"])).T
y = np.array(df_loc[["var1","var2"]])

### Basis functions

num_basis = [5**2,7**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [3]:
phi.shape

(1200, 155)

In [4]:
start_time = time.time()

In [5]:
# model layers 

input_dim = Input(shape = (phi.shape[1], ))
layer1 = Dense(100, kernel_initializer='he_uniform', activation = 'relu')(input_dim)
layer2 = Dense(100, activation = 'relu')(layer1)
layer3 = Dense(100, activation = 'relu')(layer2)
layer4 = Dense(100, activation = 'relu')(layer3)
layer5 = Dense(50, activation = 'relu')(layer4)
layer6 = Dense(50, activation = 'relu')(layer5)
final_layer = Dense(2, activation = 'linear')(layer5)

In [6]:
s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.1)
print(s_test.shape)

(120, 2)


In [7]:
s_train_ensemble, s_train_mse, X_train_ensemble, X_train_mse, y_train_ensemble, y_train_mse= train_test_split(s_train, encoder_train, y_train, test_size=0.25)
print(s_train_ensemble.shape)

(810, 2)


In [8]:
s_train_ensemble1, s_train_ensemble2, X_train_ensemble1, X_train_ensemble2, y_train_ensemble1, y_train_ensemble2= train_test_split(s_train_ensemble, X_train_ensemble, y_train_ensemble, test_size=0.3)
print(s_train_ensemble1.shape)
base_model = Model(inputs = input_dim, outputs = final_layer)
optimizer = keras.optimizers.Adam(lr=0.01)
# Compile the Model
base_model.compile(optimizer = optimizer, loss = 'mse')
base_model.summary()
base_model.fit(X_train_ensemble1, y_train_ensemble1,validation_split = 0.1, epochs = 400,
                batch_size = 64,verbose = 2)

(567, 2)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 155)]             0         
_________________________________________________________________
dense (Dense)                (None, 100)               15600     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 2)              

8/8 - 0s - loss: 0.0092 - val_loss: 0.0792
Epoch 125/400
8/8 - 0s - loss: 0.0097 - val_loss: 0.0818
Epoch 126/400
8/8 - 0s - loss: 0.0086 - val_loss: 0.0687
Epoch 127/400
8/8 - 0s - loss: 0.0097 - val_loss: 0.0707
Epoch 128/400
8/8 - 0s - loss: 0.0101 - val_loss: 0.0718
Epoch 129/400
8/8 - 0s - loss: 0.0113 - val_loss: 0.0762
Epoch 130/400
8/8 - 0s - loss: 0.0114 - val_loss: 0.1074
Epoch 131/400
8/8 - 0s - loss: 0.0085 - val_loss: 0.0584
Epoch 132/400
8/8 - 0s - loss: 0.0116 - val_loss: 0.0965
Epoch 133/400
8/8 - 0s - loss: 0.0120 - val_loss: 0.0735
Epoch 134/400
8/8 - 0s - loss: 0.0067 - val_loss: 0.1075
Epoch 135/400
8/8 - 0s - loss: 0.0070 - val_loss: 0.0660
Epoch 136/400
8/8 - 0s - loss: 0.0049 - val_loss: 0.0569
Epoch 137/400
8/8 - 0s - loss: 0.0041 - val_loss: 0.0885
Epoch 138/400
8/8 - 0s - loss: 0.0038 - val_loss: 0.0758
Epoch 139/400
8/8 - 0s - loss: 0.0034 - val_loss: 0.0590
Epoch 140/400
8/8 - 0s - loss: 0.0034 - val_loss: 0.0623
Epoch 141/400
8/8 - 0s - loss: 0.0032 - val_l

Epoch 268/400
8/8 - 0s - loss: 0.0045 - val_loss: 0.0587
Epoch 269/400
8/8 - 0s - loss: 0.0029 - val_loss: 0.0860
Epoch 270/400
8/8 - 0s - loss: 0.0031 - val_loss: 0.0679
Epoch 271/400
8/8 - 0s - loss: 0.0020 - val_loss: 0.0720
Epoch 272/400
8/8 - 0s - loss: 0.0017 - val_loss: 0.0675
Epoch 273/400
8/8 - 0s - loss: 0.0015 - val_loss: 0.0771
Epoch 274/400
8/8 - 0s - loss: 0.0013 - val_loss: 0.0857
Epoch 275/400
8/8 - 0s - loss: 0.0012 - val_loss: 0.0589
Epoch 276/400
8/8 - 0s - loss: 0.0015 - val_loss: 0.0755
Epoch 277/400
8/8 - 0s - loss: 0.0013 - val_loss: 0.0707
Epoch 278/400
8/8 - 0s - loss: 0.0011 - val_loss: 0.0692
Epoch 279/400
8/8 - 0s - loss: 0.0014 - val_loss: 0.0839
Epoch 280/400
8/8 - 0s - loss: 0.0013 - val_loss: 0.0632
Epoch 281/400
8/8 - 0s - loss: 0.0012 - val_loss: 0.0679
Epoch 282/400
8/8 - 0s - loss: 9.9240e-04 - val_loss: 0.0736
Epoch 283/400
8/8 - 0s - loss: 8.1983e-04 - val_loss: 0.0771
Epoch 284/400
8/8 - 0s - loss: 8.3353e-04 - val_loss: 0.0796
Epoch 285/400
8/8 -

In [9]:
base_model1 = Model(inputs = input_dim, outputs = layer5)
# base_model1.layers[-1].trainable = False

In [10]:
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae
# define and fit the model
def fit_model(X_train, y_train,X_test,y_test):
    # DeepKriging model for continuous data
    ensemble_model = Sequential()

    ensemble_model.add(base_model1)
#     ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(50, activation = "relu"))
    ensemble_model.add(Dense(2, activation='linear'))
    ensemble_model.layers[-3].trainable = False
    optimizer = keras.optimizers.Adam(lr=0.01)
    ensemble_model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])
    ensemble_model.fit(X_train, y_train,#validation_data=(X_test,y_test), 
                       epochs = 200, batch_size = 64, verbose = 0)
    return ensemble_model

# fit an ensemble of models
def fit_ensemble(n_members, X_train, Y_train):
    ensemble = list()
    for i in range(n_members):
#         x_train, x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size = 0.1)
        # define and fit the model on the training set
        model = fit_model(X_train, Y_train,X_train,Y_train)
        # evaluate model on the test set
#         yhat = model.predict(x_test, verbose=0)
#         mae1 = mae(yhat, y_test)
#         print('>%d, MAE: %.3f' % (i+1, mae1))
        # store the model
        print(i)
        ensemble.append(model)
    return ensemble
def y_list_uni(yhat,i):
    the_list = list()
    for j in range(len(yhat)):
        the_list.append(yhat[j][0][i])
    return the_list
# make predictions with the ensemble and calculate a prediction interval
def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / (n-1)
    return variance
def predict_with_pi(ensemble, X):
    mean_vec1 = list()
    mean_vec2 = list()
    var_vec1 = list()
    var_vec2 = list()
    # make predictions
    yhat = [model.predict(X, verbose=0) for model in ensemble]
    for data in tqdm(range(X.shape[0])):
        yhat1 = np.asarray(np.asarray(yhat)[:,data,:])
        var1_pred = yhat1[:,0]
        var2_pred = yhat1[:,1]
        mean1 = var1_pred.mean()
        var1 = variance(var1_pred)
        mean2 = var2_pred.mean()
        var2 = variance(var2_pred)
        mean_vec1.append(mean1)
        mean_vec2.append(mean2)
        var_vec1.append(var1)
        var_vec2.append(var2)
    return mean_vec1, var_vec1, mean_vec2, var_vec2

In [12]:
n_members = 20
ensemble = fit_ensemble(n_members, X_train_ensemble, y_train_ensemble)

# train data mean and variance vectors
mean_vec1, var_vec1, mean_vec2, var_vec2 = predict_with_pi(ensemble, X_train_mse)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


100%|██████████| 270/270 [00:00<00:00, 1316.27it/s]


In [13]:
# random error calculation on training data
r1 = list()
r2 = list()
r1 = (y_train_mse[:,0] - mean_vec1)**2 - var_vec1
for i in range(len(r1)):
    if r1[i] < 0: r1[i] = 0.0
r2 = (y_train_mse[:,1] - mean_vec2)**2 - var_vec2
for i in range(len(r2)):
    if r2[i] < 0: r2[i] = 0.0

In [14]:
def calc_distance(s1,s2):
    return(np.sqrt((s1[0] - s2[0])**2 + (s1[1] - s2[1])**2))

def get_nearest_data(s_train,s_test,r,k):
    dist_mat = np.zeros((len(s_test),len(s_train)))
    nearest_var = np.zeros((len(s_test)))
    for i in range(len(s_train)):
        for j in range(len(s_test)):
            dist_mat[j][i] = calc_distance(s_train[i],s_test[j])
    for i in range(len(s_test)):
        a = dist_mat[i]
        b = np.argpartition(a,k)[:k]
        y_val = []
        for index in b:
            y_val.append(r[index])
        nearest_var[i] = (np.mean(y_val))
    return nearest_var

In [15]:
# random error calculation on test data with neighbourhood approach
r1_pred = get_nearest_data(s_train_mse,s_test,r1,40)
r2_pred = get_nearest_data(s_train_mse,s_test,r2,40)

# mean and variance vector for prediction data
mean_vec1, var_vec1, mean_vec2, var_vec2 = predict_with_pi(ensemble, encoder_test)


end_time = time.time()
print("%s seconds", end_time - start_time)


100%|██████████| 120/120 [00:00<00:00, 618.04it/s]

%s seconds 798.0267617702484


In [16]:

# e = list()
# for i in range(len(r1_pred)):
#     e.append(r1_pred[i][0])
# print(len(var_vec1))
# error_vec2 = model_mse2.predict(encoder_test)
var_vec1 = np.asarray(var_vec1)
var_vec2 = np.asarray(var_vec2)
# e = np.asarray(e)
lower_bound1 = mean_vec1 - 1.96*np.sqrt(var_vec1 + r1_pred)
upper_bound1 = mean_vec1 + 1.96*np.sqrt(var_vec1 + r1_pred)
lower_bound2 = mean_vec2 - 1.96*np.sqrt(var_vec2 + r2_pred)
upper_bound2 = mean_vec2 + 1.96*np.sqrt(var_vec2 + r2_pred)

In [17]:
count_var0 = 0
count_var1 = 0
for i in range(len(y_test)):
    if ((y_test[i,0] > lower_bound1[i]) and (y_test[i,0] < upper_bound1[i])): count_var0 +=1
    if y_test[i,1] > lower_bound2[i] and y_test[i,1] < upper_bound2[i]: count_var1 +=1
    

In [18]:
picp1 = count_var0/len(y_test)
picp2 = count_var1/len(y_test)
print(str(picp1)+","+str(picp2))

0.9583333333333334,0.95


In [19]:
width1 = np.mean(upper_bound1 - lower_bound1)
width2 = np.mean(upper_bound2 - lower_bound2)
print(str(width1)+","+str(width2))

1.0243789720870606,1.0793826950773922


In [20]:
df_bounds = pd.DataFrame(np.vstack((lower_bound1,upper_bound1,lower_bound2,upper_bound2)).T,
                         columns = ["l1","u1","l2","u2"])

In [21]:
df_bounds.to_csv("../plot_results/bounds.csv")